In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from pprint import pprint
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import FreqDist


In [ ]:
#Reading of .csv file
df = pd.read_csv("TheGuardian.csv")
content = df['Text']
assert content.isnull().count() == len(content)

In [ ]:
#Before stop word removal count
count = [len(content[i]) for i in range(len(content))]
# data['Count Before'] = count
print(count)

In [ ]:
#Tokenization
from nltk.tokenize import RegexpTokenizer


data = []
tokenizer = RegexpTokenizer(r'\w+')

content = content.apply(lambda x: tokenizer.tokenize(x))
content.head(20)

In [ ]:
#Removal of Stop Words
from nltk.corpus import stopwords
set(stopwords.words('english'))

def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

content = content.apply(lambda x: remove_stopwords(x))

In [ ]:
#Lemmatization (OLD VERSION)
# lemmatizer = WordNetLemmatizer()

# def word_lemmatizer(text):`
#     lem_text = [lemmatizer.lemmatize(i, pos="v") for i in text]
#     return lem_text

# content = content.apply(lambda x: word_lemmatizer(x))

# content_freq = content

In [ ]:
#Lemmatization
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i, pos="a") for i in text]
    lem_text = [lemmatizer.lemmatize(i, pos="s") for i in text]
    lem_text = [lemmatizer.lemmatize(i, pos="v") for i in text]
    lem_text = [lemmatizer.lemmatize(i, pos="r") for i in text]
    lem_text = [lemmatizer.lemmatize(i, pos="n") for i in text]
    return lem_text

content = content.apply(lambda x: word_lemmatizer(x))
print(content[0])
content_freq = content

In [ ]:
#Count after removal of stop words
countaft = [len(content[i]) for i in range(len(content))]
print(countaft)

In [ ]:
#Tagging
content_tag = [nltk.pos_tag(i) for i in content]
print(content_tag)

In [ ]:
#Difference for the count before and after stop word removal
count_diff = []
for i in range(len(count)):
    count_diff.append(round(((count[i] - countaft[i])/count[i])*100, 1))
print(count_diff)

In [ ]:
# Freq = [FreqDist(content_freq[i]).most_common(5) for i in range(len(content_freq))]

In [ ]:
# df['Amount Cleansed (%)'] = count_diff
# df['Frequency'] = Freq

# df.to_csv('Output.csv')

In [ ]:
# word_count = 0
# word_total_count = 0
# NNP_words = []
# for articles in content:
#     word_total_count += len(articles)
#     for words in articles:
#         if words[1] == 'NNP':
#             NNP_words.append(words[0].upper())
#             word_count += 1
# print(word_count/word_total_count * 100)
# NNP_Freq = FreqDist(NNP_words).most_common(50)
# pprint(NNP_Freq)

In [ ]:
# feeling_words = []
# for articles in content:
#     word_total_count += len(articles)
#     for words in articles:
#         if words[1] == 'JJ':
#             feeling_words.append((words[0].upper()))

# Feeling_Freq = FreqDist(feeling_words).most_common(50)
# pprint(Feeling_Freq)         

In [ ]:
#Creating a main list of the the total articles with feeling words that contain the feeling words for the respective articles
total_fw = []

for articles in content_tag:
    article_fw = []
    for words in articles:
        if words[1].startswith('JJ'):
            article_fw.append(words)
    total_fw.append(article_fw)

In [ ]:
#Mapping of the tags to match the WordNet's POS 
from nltk.corpus import wordnet as wn

def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

for article in total_fw:
    for words in article:
        penn_to_wn(words[1])

In [ ]:
#To get the sentiments based on the dictionary from WordNet that reads the phrases/sentence and returns a numerical value for
#the +ve and -ve sentiments
def get_sentiment(word,tag):
    """ returns list of pos neg and objective score. But returns empty list if not present in senti wordnet. """

    wn_tag = penn_to_wn(tag)
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return []

    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

In [ ]:
#The getting of sentiments for all the words in each article
net_senti = []
for article in total_fw:
    article_senti = []
    for words in article:
        
        senti = get_sentiment(words[0], words[1])
        if len(senti) > 0:
            print([words[0], (senti[0] - senti[1])])
            article_senti.append([words[0], (senti[0] - senti[1])])
    net_senti.append(article_senti)

In [ ]:
#Final pritting and to count the number of articles that are positive, negative or neutral based on the sentiment values
article_senti = []
for article in net_senti:
    sentiment = 0
    article_word_count = 0
    for senti in article:
        print(senti[0], senti[1])
        sentiment += senti[1]
        article_word_count += 1
    print("\n********************************************\nTotal Sentiment: ", sentiment)
    print("Word Count In Article: ", article_word_count, "\n********************************************")
    print("\n**********************\n NEXT ARTICLE \n**********************")
    article_senti.append(sentiment)